In [ ]:
from mech_module import *
import numpy as np
import pandas as pd
from pyswarm import pso
import matplotlib.pyplot as plt


The model will be made in this form:
Y(t) = a(t) + b(t) + c(t) + d(t)

Where:

a(t) is auto-regressive term
a(t) = alpha * a(t-1)

b(t) is lagged dependent variable
b(t) = beta * Y(t-1)

c(t) is seasonal term
c(t) = gamma * sin(2πt/365) + delta * cos(2πt/365)

d(t) is stochastic term
d(t) ~ N(0, sigma)

Parameters estimates will be determined from actual river flow data.
alpha, beta, gamma, delta, sigma 


DATA https://nrfa.ceh.ac.uk/data/station/download?stn=15021&dt=gdf

In [ ]:
# component definitions
def a_t1(alpha, a_t):
    return alpha * a_t

def b_t1(beta, Y_t):
    return beta * Y_t

def c_t1(gamma, delta, t):
    return gamma * np.sin(2 * np.pi * t / 365) + delta * np.cos(2 * np.pi * t / 365)

def d_t(sigma):
    return np.random.normal(0, sigma)

def Y_t1(at, bt, ct, dt):
    return at + bt + ct + dt



In [ ]:


# import data
data = pd.read_csv('15021_gdf.csv')
# column names ['date', 'flow', 'nan']
# first 18 rows are descriptions
# the flow is average daily flow in m^3/s
data = data[19:]
# remove last column
data = data.iloc[:, :-1]
# name columns
data.columns = ['date', 'flow']
# convert flow to numeric
data['flow'] = pd.to_numeric(data['flow'], errors='coerce')
# convert date to datetime
data['date'] = pd.to_datetime(data['date'])

# station area width = 7.5m , depth = approx 0.5m (on visual inspection)
# https://nrfa.ceh.ac.uk/data/station/info/15021

# convert flow to velocity
data['velocity'] = data['flow'] / (7.5 * 0.5)


In [ ]:
# split the data
n = len(data)
train = data.iloc[:int(n * 0.2)]
test = data.iloc[int(n * 0.2):]

# define the objective function
def objective(params, data):
    alpha, beta, gamma, delta, sigma = params
    a = data['velocity'].iloc[0]
    b = data['velocity'].iloc[0]
    c = c_t1(gamma, delta, 1)
    d = d_t(sigma)
    Y = Y_t1(a, b, c, d)
    error = 0
    for t in range(1, len(data)):
        a = a_t1(alpha, a)
        b = b_t1(beta, Y)
        c = c_t1(gamma, delta, t)
        d = d_t(sigma)
        Y = Y_t1(a, b, c, d)
        error += (data['velocity'].iloc[t] - Y) ** 2
    return error

# initial guesses
#  0.99969635 -0.27333223 -0.01627534  0.68633705  0.12686814
alpha = 0.99969635
beta = -0.27333223
gamma = -0.01627534
delta = 0.68633705
sigma = 0.12686814
params = [alpha, beta, gamma, delta, sigma]

# perform the optimisation
result = pso(objective,[-10,-10,-10,-10,0], [10, 10, 10,10,10], args=(train,), swarmsize=100, maxiter=100, minstep=1e-8, minfunc=1e-8, debug=True)


In [ ]:
# get the parameters
alpha, beta, gamma, delta, sigma = result[0]

 plot the model against the data
a = train['velocity'].iloc[0]
b = train['velocity'].iloc[0]
c = c_t1(gamma, delta, 1)
d = d_t(sigma)
Y = Y_t1(a, b, c, d)

model = [Y]
for t in range(1, len(train)):
    a = a_t1(alpha, a)
    b = b_t1(beta, Y)
    c = c_t1(gamma, delta, t)
    d = d_t(sigma)
    Y = Y_t1(a, b, c, d)
    model.append(Y)

# adjust so that no negative values
model = np.array(model)
model[model < 0] = 0

train['model'] = model
fig = px.line(train, x='date', y=['velocity', 'model'], title='Model vs Data')
fig.show()